# Day 6: Guard Gallivant

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
```

The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.
Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:
```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```
Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:
```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```
This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):
```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```
By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:
```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```
In this example, the guard will visit 41 distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

In [91]:
def get_data(file_path: str = "data.txt"):
    # Return a 2D array of the data
    with open(file_path) as f:
        lines = f.read().splitlines()
    return [list(line) for line in lines]

In [92]:
from enum import StrEnum


class Direction(StrEnum):
    UP = "^"
    DOWN = "v"
    LEFT = "<"
    RIGHT = ">"


def turn(current: Direction) -> Direction:
    turns = {
        Direction.UP: Direction.RIGHT,
        Direction.RIGHT: Direction.DOWN,
        Direction.DOWN: Direction.LEFT,
        Direction.LEFT: Direction.UP
    }
    return turns[current]

In [93]:
def print_board(data):
    for row in data:
        print(" ".join(row))
    print()

def find_start(data):
    for i, row in enumerate(data):
        for j, cell in enumerate(row):
            if cell in [Direction.UP, Direction.DOWN, Direction.LEFT, Direction.RIGHT]:
                return i, j, cell

In [94]:
def update_board(data, x1, y1, x2, y2, direction):
    # Update the board, marking previous position with X and new position with updated direction
    data[x1][y1] = "X"
    data[x2][y2] = direction if type(direction) == str else direction.value

In [95]:
def move(start, data):
    x, y, direction = start
    visited = set()
    states = set()
    visited.add((x, y))
    states.add((x, y, direction))
    
    while True:
        start_x, start_y = x, y

        # Check if we got to the edge of the board and are moving directly into a wall
        if direction == Direction.UP and x - 1 < 0:
                break
        elif direction == Direction.DOWN and x + 1 >= len(data):
            break
        elif direction == Direction.LEFT and y - 1 < 0:
            break
        elif direction == Direction.RIGHT and y + 1 >= len(data[0]):
            break

        if direction == Direction.UP:
            if x - 1 >= 0 and data[x - 1][y] in [".", "X"]:
                x -= 1
            else:
                direction = turn(direction)
        elif direction == Direction.DOWN:
            if x + 1 < len(data) and data[x + 1][y] in [".", "X"]:
                x += 1
            else:
                direction = turn(direction)
        elif direction == Direction.LEFT:
            if y - 1 >= 0 and data[x][y - 1] in [".", "X"]:
                y -= 1
            else:
                direction = turn(direction)
        elif direction == Direction.RIGHT:
            if y + 1 < len(data[0]) and data[x][y + 1] in [".", "X"]:
                y += 1
            else:
                direction = turn(direction)
        
        # Check for loop
        if (x, y, direction) in states:
            # print("Loop detected")
            return False
        states.add((x, y, direction))
        
        visited.add((x, y))
        update_board(data, start_x, start_y, x, y, direction)
        # print_board(data)

        if x < 0 or x >= len(data) or y < 0 or y >= len(data[0]):
            break

        # time.sleep(0.01)
    return visited

def run():
    data = get_data("sample1.txt")
    start = find_start(data)
    print_board(data)
    visited = move(start, data)
    print_board(data)
    print(len(visited))

In [96]:
run()

. . . . # . . . . .
. . . . . . . . . #
. . . . . . . . . .
. . # . . . . . . .
. . . . . . . # . .
. . . . . . . . . .
. # . . ^ . . . . .
. . . . . . . . # .
# . . . . . . . . .
. . . . . . # . . .

. . . . # . . . . .
. . . . X X X X X #
. . . . X . . . X .
. . # . X . . . X .
. . X X X X X # X .
. . X . X . X . X .
. # X X X X X X X .
. X X X X X X X # .
# X X X X X X X . .
. . . . . . # v . .

41


# Part Two

While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:

```
....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...
```
Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...
```
Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...
```
Option four, put an alchemical retroencabulator near the bottom left corner:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...
```
Option five, put the alchemical retroencabulator a bit to the right instead:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...
```
Option six, put a tank of sovereign glue right next to the tank of universal solvent:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..
```

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. How many different positions could you choose for this obstruction?

## Solution
Brute force: we can modify the original move function to detect loops. Next we simply need to add obstructions to every spot on the board where the guard has visited and see if the guard gets stuck in a loop. 

In [97]:
def get_next_board(previously_visited, filename="data.txt"):
    # This function returns a generator that yields a new board with an obstruction at every position where the guard had visited from the previous step (visited will contain a tuple of x, y coordinates where he visited before)
    board_data = get_data(filename)
    
    for (x, y) in previously_visited:
        new_board = [row.copy() for row in board_data]
        new_board[x][y] = "O"
        yield new_board

In [101]:
def find_loops(start, data, filename):
    visited = move(start, data)
    loops = 0
    for new_board in get_next_board(visited, filename):
        # print_board(new_board)
        if move(start, new_board) is False:
            # print("Loop detected")
            loops += 1
    return loops

def run(filename="data.txt"):
    data = get_data(filename)
    start = find_start(data)
    loops = find_loops(start, data, filename)
    print(loops)

In [102]:
run()

1586
